# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-23-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-23-2020'

In [108]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-24 04:27:56,33.93911,67.709953,37999,1387,28180,8432.0,Afghanistan,97.612688,3.650096
1,NaN,NaN,NaN,Albania,2020-08-24 04:27:56,41.15330,20.168300,8427,250,4332,3845.0,Albania,292.827855,2.966655
2,NaN,NaN,NaN,Algeria,2020-08-24 04:27:56,28.03390,1.659600,41460,1435,29142,10883.0,Algeria,94.547352,3.461167
3,NaN,NaN,NaN,Andorra,2020-08-24 04:27:56,42.50630,1.521800,1045,53,875,117.0,Andorra,1352.488190,5.071770
4,NaN,NaN,NaN,Angola,2020-08-24 04:27:56,-11.20270,17.873900,2171,96,818,1257.0,Angola,6.605557,4.421925


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37431,37551,37596,37599,37599,37599,37856,37894,37953,37999
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7117,7260,7380,7499,7654,7812,7967,8119,8275,8427
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,37664,38133,38583,39025,39444,39847,40258,40667,41068,41460


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1363,1370,1375,1375,1375,1375,1385,1385,1385,1387
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,219,225,228,230,232,234,238,240,245,250
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1351,1360,1370,1379,1391,1402,1411,1418,1424,1435


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,26714,27166,27166,27166,27166,27166,27681,28016,28016,28180
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3695,3746,3794,3816,3871,3928,3986,4096,4184,4332
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,26308,26644,27017,27347,27653,27971,28281,28587,28874,29142


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1196,1206,1218,1219,1235,1241,1240,1255,1264,1266
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3791,3839,3877,3888,3906,3931,3957,3997,4037,4069
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,590,598,604,605,606,613,619,622,626,629


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,22,22,22,22,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,29,29,29,29,30,30,31,32,32,32
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6,6,6,6,6,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-24 04:27:56,32.539527,-86.644082,1266,22,0,1244.0,"Autauga, Alabama, US",2266.015143,1.737757
668,1075.0,Lamar,Alabama,US,2020-08-24 04:27:56,33.779950,-88.096680,260,2,0,258.0,"Lamar, Alabama, US",1883.375589,0.769231
669,1077.0,Lauderdale,Alabama,US,2020-08-24 04:27:56,34.901719,-87.656247,1329,22,0,1307.0,"Lauderdale, Alabama, US",1433.208597,1.655380


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5701679,176802,1997761
Brazil,3605783,114744,2947784
India,3106348,57542,2338035


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5701679,176802,1997761,3527116,2020-08-24 04:27:56,37.919625,-91.366793
Brazil,3605783,114744,2947784,543255,2020-08-24 04:27:56,-12.669522,-48.480493
India,3106348,57542,2338035,710771,2020-08-24 04:27:56,23.088275,81.806127
Russia,954328,16341,768906,169081,2020-08-24 04:27:56,54.546312,62.120860
South Africa,609773,13059,506470,90244,2020-08-24 04:27:56,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,669280,12153,0
Florida,600571,10325,0
Texas,594173,11699,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,669280,12153,0,657127,2020-08-24 04:27:56,37.843962,-120.728594
Florida,600571,10325,0,590246,2020-08-24 04:27:56,28.940755,-82.700744
Texas,594173,11699,0,582474,2020-08-24 04:27:56,31.660643,-98.653069
New York,429737,32883,0,396854,2020-08-24 04:27:56,42.671593,-75.579694
Georgia,253949,5132,0,248817,2020-08-24 04:27:56,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,232120,23658,0
California,Los Angeles,231695,5545,0
Florida,Miami-Dade,151986,2243,0
Arizona,Maricopa,131787,2790,0
Illinois,Cook,121308,5013,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,232120,23658,0,208462,2020-08-24 04:27:56,40.767273,-73.971526,36061.0
California,Los Angeles,231695,5545,0,226150,2020-08-24 04:27:56,34.308284,-118.228241,6037.0
Florida,Miami-Dade,151986,2243,0,149743,2020-08-24 04:27:56,25.611236,-80.551706,12086.0
Arizona,Maricopa,131787,2790,0,128997,2020-08-24 04:27:56,33.348359,-112.491815,4013.0
Illinois,Cook,121308,5013,0,116295,2020-08-24 04:27:56,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,37894,8119,40667,1045,2068,94,329043,42477,24602,24762,...,325263,1516,38074,38219,1009,18313,10,1906,10627,5815
2020-08-22,37953,8275,41068,1045,2134,94,336802,42616,24811,25062,...,326595,1521,38532,38957,1014,18476,10,1907,10831,5893
2020-08-23,37999,8427,41460,1045,2171,94,342154,42792,24915,25253,...,327643,1527,38946,39564,1016,18802,10,1911,11082,5930


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,1385,240,1418,53,94,3,6730,842,485,730,...,41491,42,260,317,25,122,1,542,277,152
2020-08-22,1385,245,1424,53,94,3,6848,850,502,732,...,41509,42,267,323,26,125,1,546,279,153
2020-08-23,1387,250,1435,53,96,3,6985,852,517,732,...,41515,42,273,329,27,128,1,553,280,155


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,28016,4096,28587,875,804,89,239806,35693,18458,21260,...,1544,1249,33989,27306,545,11102,8,1058,9126,4587
2020-08-22,28016,4184,28874,875,814,89,245781,35907,18758,21406,...,1547,1264,34576,28453,563,11103,8,1066,9942,4629
2020-08-23,28180,4332,29142,875,818,89,251400,35991,19231,21558,...,1547,1276,34987,29966,568,11843,8,1086,9942,4872


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1196,1206,1218,1219,1235,1241,1240,1255,1264,1266
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,237,239,240,245,247,249,255,259,259,260
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1219,1231,1238,1240,1269,1278,1287,1308,1320,1329
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,364,367,370,370,377,379,383,389,396,402
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2735,2760,2776,2779,2861,2888,2899,2908,2916,2930


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-21,113632,4588,0,196899,55652,658900,54211,51519,16770,49,...,140844,585632,48445,1541,932,110857,69779,9077,69059,3524
2020-08-22,114532,4677,0,197895,56199,664435,54864,51519,16828,49,...,142083,589349,48814,1553,932,112068,70595,9185,70009,3543
2020-08-23,115060,4741,0,198103,56574,669280,55123,51519,16895,49,...,143937,594173,49115,1557,932,112960,71012,9273,70462,3578


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-21        1255    3997     622  501    930     528    802    2105   
2020-08-22        1264    4037     626  503    931     527    802    2134   
2020-08-23        1266    4069     629  510    934     527    804    2144   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-21          865      324  ...     156      7      112       44   
2020-08-22          869      330  ...     156      7      113       46   
2020-08-23          870      331  ...     158      7      117       48   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-21            288   399   283          0      106     11  
2020-08-22            290   401   283          0      107     11  
2020-08-23            290   409   283          0      108     11  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-21,1996,30,0,4688,663,11946,1910,4460,600,0,...,1549,11418,383,58,10,2436,1850,171,1068,37
2020-08-22,2011,31,0,4756,674,12131,1918,4460,600,0,...,1563,11581,385,58,10,2443,1857,177,1081,37
2020-08-23,2013,32,0,4771,687,12153,1918,4460,600,0,...,1567,11699,385,58,10,2467,1863,179,1081,37


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-21          22      32       7    6      6      13     36      23   
2020-08-22          22      32       7    6      6      13     36      23   
2020-08-23          22      32       7    6      6      13     36      23   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-21           38        9  ...       0      0        0        0   
2020-08-22           38        9  ...       0      0        0        0   
2020-08-23           38        9  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-21              0     0     0         36        0      0  
2020-08-22              0     0     0         36        0      0  
2020-08-23              0     0     0         36        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,0.001004,0.019079,0.010159,0.020508,0.011742,0.0,0.025427,0.003734,0.007990,0.013548,...,0.003270,0.006640,0.014036,0.017356,0.001986,0.018011,0.0,0.003686,0.024585,0.012185
2020-08-22,0.001557,0.019214,0.009861,0.000000,0.031915,0.0,0.023581,0.003272,0.008495,0.012115,...,0.004095,0.003298,0.012029,0.019310,0.004955,0.008901,0.0,0.000525,0.019196,0.013414
2020-08-23,0.001212,0.018369,0.009545,0.000000,0.017338,0.0,0.015891,0.004130,0.004192,0.007621,...,0.003209,0.003945,0.010744,0.015581,0.001972,0.017645,0.0,0.002098,0.023174,0.006279


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,0.000000,0.008403,0.004961,0.0,0.010753,0.0,0.032684,0.007177,0.027542,0.001372,...,0.000048,0.02439,0.031746,0.019293,0.000000,0.016667,0.0,0.001848,0.010949,0.006623
2020-08-22,0.000000,0.020833,0.004231,0.0,0.000000,0.0,0.017533,0.009501,0.035052,0.002740,...,0.000434,0.00000,0.026923,0.018927,0.040000,0.024590,0.0,0.007380,0.007220,0.006579
2020-08-23,0.001444,0.020408,0.007725,0.0,0.021277,0.0,0.020006,0.002353,0.029880,0.000000,...,0.000145,0.00000,0.022472,0.018576,0.038462,0.024000,0.0,0.012821,0.003584,0.013072


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,0.012102,0.027597,0.010820,0.0,0.083558,0.0,0.026343,0.006117,0.033946,0.007917,...,0.000000,0.005636,0.021887,0.037068,0.005535,0.039318,0.0,0.000000,0.000000,0.013702
2020-08-22,0.000000,0.021484,0.010040,0.0,0.012438,0.0,0.024916,0.005996,0.016253,0.006867,...,0.001943,0.012010,0.017270,0.042005,0.033028,0.000090,0.0,0.007561,0.089415,0.009156
2020-08-23,0.005854,0.035373,0.009282,0.0,0.004914,0.0,0.022862,0.002339,0.025216,0.007101,...,0.000000,0.009494,0.011887,0.053175,0.008881,0.066649,0.0,0.018762,0.000000,0.052495


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-21,0.01052,0.015044,NaN,0.003154,0.016196,0.010169,0.000000,0.001692,0.003110,0.0,...,0.011992,0.009898,0.009649,0.002602,0.072497,0.008901,0.005620,0.010014,0.012106,0.019381
2020-08-22,0.00792,0.019398,NaN,0.005058,0.009829,0.008400,0.012046,0.000000,0.003459,0.0,...,0.008797,0.006347,0.007617,0.007787,0.000000,0.010924,0.011694,0.011898,0.013756,0.005392
2020-08-23,0.00461,0.013684,NaN,0.001051,0.006673,0.007292,0.004721,0.000000,0.003981,0.0,...,0.013049,0.008185,0.006166,0.002576,0.000000,0.007959,0.005907,0.009581,0.006471,0.009879


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-21      0.012097  0.010109  0.004847  0.014170  0.019737  0.001898   
2020-08-22      0.007171  0.010008  0.006431  0.003992  0.001075 -0.001894   
2020-08-23      0.001582  0.007927  0.004792  0.013917  0.003222  0.000000   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-21      0.007538  0.015927  0.004646  0.015674  ...  0.019608    0.0   
2020-08-22      0.000000  0.013777  0.004624  0.018519  ...  0.000000    0.0   
2020-08-23      0.002494  0.004686  0.001151  0.003030  ...  0.012821    0.0   

Province_State                                                            \
Admin2          Sheridan  Sublette Sweetwater     Teton Uinta Unassigned   
2020-08-21      0.037037  0.073171   0.010526  0.002513   0.0        NaN   
2020-08-22      0.008929  0.045455   0.006944  0.005013   0.0        NaN   
2020-08-23      0.035398  0.043478   0.000000  0.019950   0.0        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-08-21      0.000000    inf  
2020-08-22      0.009434    0.0  
2020-08-23      0.009346    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-21,0.011145,0.034483,NaN,0.000854,0.034321,0.011516,0.003678,0.000449,0.008403,NaN,...,0.040995,0.021836,0.005249,0.0,0.0,0.003708,0.007077,0.023952,0.000937,0.088235
2020-08-22,0.007515,0.033333,NaN,0.014505,0.016591,0.015486,0.004188,0.000000,0.000000,NaN,...,0.009038,0.014276,0.005222,0.0,0.0,0.002874,0.003784,0.035088,0.012172,0.000000
2020-08-23,0.000995,0.032258,NaN,0.003154,0.019288,0.001814,0.000000,0.000000,0.000000,NaN,...,0.002559,0.010189,0.000000,0.0,0.0,0.009824,0.003231,0.011299,0.000000,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-21         0.0  0.032258     0.0  0.0    0.2     0.0    0.0     0.0   
2020-08-22         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2020-08-23         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-21          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-22          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-23          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-21            NaN   NaN   NaN   0.090909      NaN    NaN  
2020-08-22            NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-23            NaN   NaN   NaN   0.000000      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,0.002266,0.019458,0.010350,0.013179,0.014582,0.001387,0.024617,0.004449,0.008277,0.013025,...,0.003281,0.007950,0.014701,0.021245,0.006467,0.019606,4.818676e-20,0.003548,0.022241,0.018426
2020-08-22,0.001912,0.019336,0.010105,0.006590,0.023249,0.000693,0.024099,0.003861,0.008386,0.012570,...,0.003688,0.005624,0.013365,0.020277,0.005711,0.014253,2.409338e-20,0.002037,0.020719,0.015920
2020-08-23,0.001562,0.018852,0.009825,0.003295,0.020293,0.000347,0.019995,0.003995,0.006289,0.010096,...,0.003448,0.004785,0.012055,0.017929,0.003842,0.015949,1.204669e-20,0.002067,0.021946,0.011099


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,0.001945,0.010984,0.006046,0.000019,0.012985,1.480847e-39,0.033274,0.005566,0.026754,0.000766,...,0.000137,0.020196,0.025974,0.022352,0.000449,0.018420,0.0,0.002922,0.013299,0.017082
2020-08-22,0.000972,0.015909,0.005139,0.000009,0.006492,7.404237e-40,0.025404,0.007534,0.030903,0.001753,...,0.000285,0.010098,0.026449,0.020640,0.020225,0.021505,0.0,0.005151,0.010260,0.011831
2020-08-23,0.001208,0.018158,0.006432,0.000005,0.013884,3.702118e-40,0.022705,0.004943,0.030392,0.000876,...,0.000215,0.005049,0.024460,0.019608,0.029343,0.022752,0.0,0.008986,0.006922,0.012451


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-21,0.010941,0.020883,0.011121,0.001129,0.067659,0.004142,0.025270,0.006552,0.065841,0.006678,...,0.008620,0.007658,0.020608,0.036724,0.017958,0.034817,2.823443e-22,0.002801,0.008426,0.053323
2020-08-22,0.005470,0.021184,0.010580,0.000565,0.040048,0.002071,0.025093,0.006274,0.041047,0.006773,...,0.005282,0.009834,0.018939,0.039365,0.025493,0.017454,1.411722e-22,0.005181,0.048920,0.031240
2020-08-23,0.005662,0.028278,0.009931,0.000282,0.022481,0.001036,0.023977,0.004307,0.033132,0.006937,...,0.002641,0.009664,0.015413,0.046270,0.017187,0.042051,7.058608e-23,0.011972,0.024460,0.041867


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-21,0.009925,0.016052,NaN,0.003431,0.013300,0.010312,0.002775,0.001792,0.003639,9.317144e-49,...,0.011501,0.010165,0.008975,0.002734,0.058299,0.008279,0.007066,0.012565,0.011836,0.015704
2020-08-22,0.008923,0.017725,NaN,0.004245,0.011564,0.009356,0.007410,0.000896,0.003549,4.658572e-49,...,0.010149,0.008256,0.008296,0.005261,0.029149,0.009602,0.009380,0.012232,0.012796,0.010548
2020-08-23,0.006766,0.015704,NaN,0.002648,0.009119,0.008324,0.006066,0.000448,0.003765,2.329286e-49,...,0.011599,0.008221,0.007231,0.003918,0.014575,0.008781,0.007643,0.010906,0.009633,0.010213


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-21      0.007563  0.008231  0.006783  0.013583  0.017011  0.008916   
2020-08-22      0.007367  0.009119  0.006607  0.008787  0.009043  0.003511   
2020-08-23      0.004475  0.008523  0.005700  0.011352  0.006133  0.001756   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-21      0.004168  0.018405  0.003275  0.014939  ...  0.016321   
2020-08-22      0.002084  0.016091  0.003950  0.016729  ...  0.008161   
2020-08-23      0.002289  0.010388  0.002550  0.009880  ...  0.010491   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-21      0.010807  0.055273  0.039736   0.008653  0.004975  0.002546   
2020-08-22      0.005404  0.032101  0.042595   0.007799  0.004994  0.001273   
2020-08-23      0.002702  0.033750  0.043037   0.003899  0.012472  0.000637   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-21           -1.0  0.010175 -0.359887  
2020-08-22           -1.0  0.009805 -0.119962  
2020-08-23           -1.0  0.009575 -0.051412  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-21,0.010831,0.019803,NaN,0.006606,0.025657,0.011877,0.002456,0.000334,0.004629,NaN,...,0.031766,0.021491,0.009105,2.698268e-07,0.027780,0.004754,0.007691,0.017676,0.004811,0.060870
2020-08-22,0.009173,0.026568,NaN,0.010556,0.021124,0.013682,0.003322,0.000167,0.002314,NaN,...,0.020402,0.017883,0.007163,1.349134e-07,0.013890,0.003814,0.005737,0.026382,0.008491,0.030435
2020-08-23,0.005084,0.029413,NaN,0.006855,0.020206,0.007748,0.001661,0.000083,0.001157,NaN,...,0.011481,0.014036,0.003582,6.745670e-08,0.006945,0.006819,0.004484,0.018841,0.004246,0.015218


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-21      0.000024  0.026704  0.020931  0.050018  0.100143  0.005682   
2020-08-22      0.000012  0.013352  0.010466  0.025009  0.050071  0.002841   
2020-08-23      0.000006  0.006676  0.005233  0.012505  0.025036  0.001420   

Province_State                                                  ... Wyoming  \
Admin2                Butler   Calhoun      Chambers  Cherokee  ...    Park   
2020-08-21      1.744336e-06  0.009389  5.822774e-10  0.031285  ...     NaN   
2020-08-22      8.721680e-07  0.004694  2.911387e-10  0.015642  ...     NaN   
2020-08-23      4.360840e-07  0.002347  1.455693e-10  0.007821  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-08-21        NaN      NaN      NaN        NaN   NaN   NaN   0.062785   
2020-08-22        NaN      NaN      NaN        NaN   NaN   NaN   0.031393   
2020-08-23        NaN      NaN      NaN        NaN   NaN   NaN   0.015696   

Province_State                  
Admin2         Washakie Weston  
2020-08-21          NaN    NaN  
2020-08-22          NaN    NaN  
2020-08-23          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200823,AK,5558,321882,NaN,48.0,NaN,NaN,NaN,7.0,...,327440,1,0,52c1365333d039e7da975f0aadb01b461bad7575,0,0,0,0,0,NaN
1,20200823,AL,115060,806609,NaN,1093.0,13502.0,NaN,1348.0,NaN,...,921669,2,0,6ec6c657edc62c73e0e7a56e191c227b5eb1e180,0,0,0,0,0,NaN
2,20200823,AR,56574,609126,NaN,500.0,3920.0,NaN,NaN,110.0,...,665700,13,12,b43e81edc09d50e99b56d5dbfe85fdc946371ea8,0,0,0,0,0,NaN
3,20200823,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,1e26b10f04c9e417c0858888c8f67d61d22674d3,0,0,0,0,0,NaN
4,20200823,AZ,198103,952766,NaN,969.0,21241.0,329.0,NaN,191.0,...,1150869,15,-30,c03b4ee84d715f897f1e88162997b8d742387b9b,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200823,AK,5558.0,321882.0,NaN,48.0,NaN,NaN,NaN,7.0,...,327440,1,0,52c1365333d039e7da975f0aadb01b461bad7575,0,0,0,0,0,NaN
1,20200823,AL,115060.0,806609.0,NaN,1093.0,13502.0,NaN,1348.0,NaN,...,921669,2,0,6ec6c657edc62c73e0e7a56e191c227b5eb1e180,0,0,0,0,0,NaN
2,20200823,AR,56574.0,609126.0,NaN,500.0,3920.0,NaN,NaN,110.0,...,665700,13,12,b43e81edc09d50e99b56d5dbfe85fdc946371ea8,0,0,0,0,0,NaN
3,20200823,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,1e26b10f04c9e417c0858888c8f67d61d22674d3,0,0,0,0,0,NaN
4,20200823,AZ,198103.0,952766.0,NaN,969.0,21241.0,329.0,NaN,191.0,...,1150869,15,-30,c03b4ee84d715f897f1e88162997b8d742387b9b,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-23,AK,5558,321882,NaN,48.0,NaN,NaN,NaN,7.0,NaN,...,327440,1,0,52c1365333d039e7da975f0aadb01b461bad7575,0,0,0,0,0,NaN
2020-08-23,AL,115060,806609,NaN,1093.0,13502.0,NaN,1348.0,NaN,734.0,...,921669,2,0,6ec6c657edc62c73e0e7a56e191c227b5eb1e180,0,0,0,0,0,NaN
2020-08-23,AR,56574,609126,NaN,500.0,3920.0,NaN,NaN,110.0,502.0,...,665700,13,12,b43e81edc09d50e99b56d5dbfe85fdc946371ea8,0,0,0,0,0,NaN
2020-08-23,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,1e26b10f04c9e417c0858888c8f67d61d22674d3,0,0,0,0,0,NaN
2020-08-23,AZ,198103,952766,NaN,969.0,21241.0,329.0,NaN,191.0,NaN,...,1150869,15,-30,c03b4ee84d715f897f1e88162997b8d742387b9b,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-23,AK,5558.0,321882.0,NaN,48.0,NaN,NaN,NaN,7.0,NaN,...,327440,1,0,52c1365333d039e7da975f0aadb01b461bad7575,0,0,0,0,0,NaN
2020-08-23,AL,115060.0,806609.0,NaN,1093.0,13502.0,NaN,1348.0,NaN,734.0,...,921669,2,0,6ec6c657edc62c73e0e7a56e191c227b5eb1e180,0,0,0,0,0,NaN
2020-08-23,AR,56574.0,609126.0,NaN,500.0,3920.0,NaN,NaN,110.0,502.0,...,665700,13,12,b43e81edc09d50e99b56d5dbfe85fdc946371ea8,0,0,0,0,0,NaN
2020-08-23,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,1e26b10f04c9e417c0858888c8f67d61d22674d3,0,0,0,0,0,NaN
2020-08-23,AZ,198103.0,952766.0,NaN,969.0,21241.0,329.0,NaN,191.0,NaN,...,1150869,15,-30,c03b4ee84d715f897f1e88162997b8d742387b9b,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE